In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:

from __future__ import print_function
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import Normalizer

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import pandas as pd

from tensorflow.keras.layers import BatchNormalization

from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.convolutional import Conv1D,MaxPooling1D
from keras.layers import Dense, Dropout,Flatten, Activation, Embedding
from keras.layers import LSTM, SimpleRNN, GRU
from keras.datasets import imdb
from keras.utils.np_utils import to_categorical
from sklearn.metrics import(precision_score, recall_score, f1_score, accuracy_score, mean_squared_error, mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import Normalizer
import h5py
from keras import callbacks
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
model_path = '/content/drive/MyDrive/Colab Notebooks/CSCI 356 Term Project/Moderate CNN Model/results/cnn_moderate_mawi3.h5'

In [4]:
#prepare callbacks
callbacks = [
    EarlyStopping(
        monitor='val_acc',
        patience=10,
        mode = 'max',
        verbose=1),
    ModelCheckpoint(model_path,
        monitor = 'val_acc',
        save_best_only = True,
        mode = 'max',
        verbose=0)
            
             

]

In [5]:
#get dataset
class dataset:
  mawi_001_train = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/CSCI 356 Term Project/MAWILab Data Preprocessing/preprocessed_mawi_train_2labels(001).pkl')
  mawi_004_test = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/CSCI 356 Term Project/MAWILab Data Preprocessing/preprocessed_mawi_test_2labels(004).pkl')

In [6]:
from sklearn import model_selection as ms
from sklearn import preprocessing as pp

In [9]:
class preprocess:
  output_columns_2labels = ["status_anomaly", "status_normal"]
  x_input = dataset.mawi_001_train.drop(output_columns_2labels, axis =1)
  y_output = dataset.mawi_001_train.loc[:, output_columns_2labels]

  x_test_input = dataset.mawi_004_test.drop(output_columns_2labels, axis=1)
  y_test = dataset.mawi_004_test.loc[:, output_columns_2labels]

  ss = pp.StandardScaler()

  x_train = ss.fit_transform(x_input) 
  x_test = ss.fit_transform(x_test_input)
  y_train = y_output.values
  y_test = y_test.values

  print("x_train.shape{}".format(x_train.shape))
  print("x_test.shape{}".format(x_test.shape))
  print("y_train.shape{}".format(y_train.shape))
  print("y_test.shape{}".format(y_test.shape))

x_train.shape(40934, 4)
x_test.shape(42599, 4)
y_train.shape(40934, 2)
y_test.shape(42599, 2)


In [10]:
#reshape the dataset
X_train = np.reshape(preprocess.x_train, (preprocess.x_train.shape[0], preprocess.x_train.shape[1],1))

X_test = np.reshape(preprocess.x_test, (preprocess.x_test.shape[0], preprocess.x_test.shape[1],1))

print("train shape after reshaping: {}". format(X_train.shape))
print("train shape after reshaping: {}". format(X_test.shape))

train shape after reshaping: (40934, 4, 1)
train shape after reshaping: (42599, 4, 1)


In [11]:
#define hyper parameters
batch_size = 64
num_classes = 2
epochs = 5
droprate = 0.5
filter_size = 3

In [12]:
#Start Neural Network 
model = Sequential() 

#convolution 1st layer 
model.add(Conv1D(64, kernel_size=(filter_size), padding="same", 
                 activation='relu', 
                 input_shape=(4, 1)))
model.add(BatchNormalization()) 
model.add(Dropout(droprate)) 
#convolution 2nd layer
model.add(Conv1D(128, kernel_size=(filter_size), activation='relu' , padding= "same"))
model.add(BatchNormalization()) 
model.add(Dropout(droprate)) 
model.add(MaxPooling1D(strides=1)) 

#FCN layer 
model.add(Flatten()) 
model.add(Dense(64,use_bias=False)) 
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(droprate)) 
#FCN layer 
model.add(Dense(32,use_bias=False)) 
model.add(BatchNormalization()) 
model.add(Activation('relu')) 
model.add(Dropout(droprate)) 

model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(loss="binary_crossentropy", optimizer="Adam", metrics=['accuracy'])

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 4, 64)             256       
                                                                 
 batch_normalization (BatchN  (None, 4, 64)            256       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 4, 64)             0         
                                                                 
 conv1d_1 (Conv1D)           (None, 4, 128)            24704     
                                                                 
 batch_normalization_1 (Batc  (None, 4, 128)           512       
 hNormalization)                                                 
                                                  

In [13]:
history = model.fit(X_train, preprocess.y_train,
                    batch_size= batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data = (X_test, preprocess.y_test), shuffle=True, callbacks = callbacks)
score = model.evaluate(X_test, preprocess.y_test, verbose=0)
print("Train loss:", score[0])
print("Training accuracy:", score[1])



Train on 40934 samples, validate on 42599 samples
Epoch 1/5
40934/40934 [==============================] - ETA: 0s - loss: 0.4292 - acc: 0.8124

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


40934/40934 [==============================] - 22s 527us/sample - loss: 0.4292 - acc: 0.8124 - val_loss: 0.4798 - val_acc: 0.6970
Epoch 2/5
40934/40934 [==============================] - 8s 196us/sample - loss: 0.3766 - acc: 0.8280 - val_loss: 0.4800 - val_acc: 0.6971
Epoch 3/5
40934/40934 [==============================] - 8s 201us/sample - loss: 0.3731 - acc: 0.8279 - val_loss: 0.4813 - val_acc: 0.6972
Epoch 4/5
40934/40934 [==============================] - 8s 201us/sample - loss: 0.3701 - acc: 0.8288 - val_loss: 0.4807 - val_acc: 0.6975
Epoch 5/5
40934/40934 [==============================] - 8s 193us/sample - loss: 0.3653 - acc: 0.8306 - val_loss: 0.4879 - val_acc: 0.6971
Train loss: 0.48791736404521746
Training accuracy: 0.6970821


In [14]:
#confusion matrix

from sklearn.metrics import classification_report, confusion_matrix

y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)

predict_proba = model.predict(X_test)
predict_classes = np.argmax(predict_proba, axis=1)

target_name=["is_Attack", "is_Normal"]

print(classification_report(np.argmax(preprocess.y_test, axis=1), y_pred, target_names=target_name, digits=4))
print(confusion_matrix(np.argmax(preprocess.y_test, axis=1), y_pred))


/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


              precision    recall  f1-score   support

   is_Attack     0.6043    0.0302    0.0576     13040
   is_Normal     0.6985    0.9913    0.8195     29559

    accuracy                         0.6971     42599
   macro avg     0.6514    0.5107    0.4385     42599
weighted avg     0.6697    0.6971    0.5863     42599

[[  394 12646]
 [  258 29301]]


In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 4, 64)             256       
                                                                 
 batch_normalization (BatchN  (None, 4, 64)            256       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 4, 64)             0         
                                                                 
 conv1d_1 (Conv1D)           (None, 4, 128)            24704     
                                                                 
 batch_normalization_1 (Batc  (None, 4, 128)           512       
 hNormalization)                                                 
                                                                 
 dropout_1 (Dropout)         (None, 4, 128)            0